In [ ]:
import json, os
import pandas as pd
from matplotlib import pyplot as plt
from collections import defaultdict
import numpy as np
from itertools import combinations
from sklearn.linear_model import LinearRegression, RANSACRegressor
from sklearn.decomposition import PCA
from wpca import WPCA
from sklearn.preprocessing import StandardScaler
from aquabyte.accuracy_metrics import AccuracyMetricsGenerator
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.optics import euclidean_distance, pixel2world
from aquabyte.visualize import Visualizer, _normalize_world_keypoints
import random
from scipy.stats import norm
from PIL import Image, ImageDraw
from urllib.parse import urlparse
from multiprocessing import Pool
import datetime as dt
import pytz
import matplotlib.dates as mdates
from mpl_toolkits.mplot3d import Axes3D
myFmt = mdates.DateFormatter('%d')

import matplotlib.cm as cm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

<h1> Load Data </h1>

In [ ]:
f = '/root/data/temp/results_452623dd3e5172ef6717c3f8e302a52c19e156b5_model_15K_4eig_hypural_plate_research-exp-id-01-vikingfjord-20190628-20190630.h5'
key = 'table'
df = pd.read_hdf(f, key)

In [ ]:
def get_world_keypoints(row):
    if 'leftCrop' in row.keypoints and 'rightCrop' in row.keypoints:
        return pixel2world(row.keypoints['leftCrop'], row.keypoints['rightCrop'], row.camera_metadata)
    else:
        return None
    
df['world_keypoints'] = df.apply(
    lambda x: get_world_keypoints(x), axis=1
)

<h1> Generate RMS Errors for best fit plane through keypoints </h1>

In [ ]:
i = 0
df['score'] = np.nan
df['max_error'] = np.nan

scores, max_errors = [], []
for idx, row in df.iterrows():
    try:
        X, y = [], []
        norm_wkps = _normalize_world_keypoints(row.world_keypoints)
        for norm_wkp in norm_wkps.values():
            X.append([
                norm_wkp[0],
                norm_wkp[2]
            ])
            y.append(norm_wkp[1])

        X, y = np.array(X), np.array(y)
        reg = LinearRegression().fit(X, y)
        score = reg.score(X, y)
        max_error = np.abs(reg.predict(X) - y).max()
        scores.append(score)
        max_errors.append(max_error)
    except Exception as e:
        print(e)
        scores.append(None)
        max_errors.append(None)

df['score'] = scores
df['max_error'] = max_errors
        

In [ ]:
%matplotlib inline
plt.figure(figsize=(20, 10))
plt.hist(df.max_error)
plt.grid()
plt.show()

In [ ]:
%matplotlib inline
hard_outlier_mask = (df.estimated_biomass_g < 0) | (df.estimated_biomass_g > 20000)
soft_outlier_mask = df.max_error > 0.05
plt.figure(figsize=(20, 10))
plt.hist(df[~hard_outlier_mask].estimated_biomass_g, bins=20, color='blue', alpha=0.5)
plt.hist(df[~soft_outlier_mask & ~hard_outlier_mask].estimated_biomass_g, bins=20, color='red', alpha=0.5)
plt.grid()
plt.show()


In [ ]:
df[~hard_outlier_mask].estimated_biomass_g.mean()

In [ ]:
df[~hard_outlier_mask & ~soft_outlier_mask].estimated_biomass_g.mean()

<h1> Visualize Bad Cases </h1>

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))

In [ ]:
v = Visualizer(rds_access_utils=rds_access_utils, s3_access_utils=s3_access_utils)

In [ ]:
# show some bad cases

df[~hard_outlier_mask & ~soft_outlier_mask & (df.estimated_biomass_g > 13000)].head(10)

In [ ]:
%matplotlib inline
kpid = 803494
v.load_data(kpid)
v.display_crops(overlay_keypoints=True, show_labels=True)

In [ ]:
%matplotlib notebook
v.display_3d_keypoints()

In [ ]:
x = df[df.id == 798364].keypoints.iloc[0]

In [ ]:
wkp = pixel2world(x['leftCrop'], x['rightCrop'], x['cameraParameters'])

In [ ]:
x['leftCrop'][0]

In [ ]:
x['rightCrop'][0]

In [ ]:
x['leftCrop'][9]

In [ ]:
x['rightCrop'][9]

In [ ]:
euclidean_distance(wkp['DORSAL_FIN'], wkp['PELVIC_FIN'])

In [ ]:
cp = x['cameraParameters']

In [ ]:
(cp['focalLengthPixel'] * cp['baseline']) / (x['leftCrop'][2]['xFrame'] - x['rightCrop'][2]['xFrame'])

In [ ]:
wkp

In [ ]:
cp